# Notebook 1 – Extract & Transform 
 


In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", None)

## Extracting and Inspecting the Dataset

In [2]:
raw_path = "/Users/nathanomenge/Desktop/Projects/Learning/DataMining_GroupProject/data/raw/budgetwise_synthetic_dirty_raw.csv"

df_raw = pd.read_csv(raw_path)
df_raw.head()

,transaction_id,user_id,date,transaction_type,category,amount,payment_mode,location,notes
0,T03512,U039,December 22 2021,Expense,Rent,998,Cash,Pune,Paid electricity bill
1,T03261,U179,03/24/2022,Expense,Food,$143,Card,Delhi,Grocery shopping
2,T04316,U143,October 18 2022,Expense,Rent,149,Cash,Bengaluru,NaN
3,T05649,U079,12/12/2021,Expense,Rent,49,UPI,NaN,Paid electricity bill
4,T14750,U020,NaN,Income,Other Income,"83,802",Bank Transfer,Chennai,Gift via app


## Inspecting the Data
### Shape and Column names

In [3]:
df_raw.shape, df_raw.columns

((15836, 9),
 Index(['transaction_id', 'user_id', 'date', 'transaction_type', 'category',
        'amount', 'payment_mode', 'location', 'notes'],
       dtype='object'))

### Data types & non-null counts 

In [4]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15836 entries, 0 to 15835
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   transaction_id    15836 non-null  object
 1   user_id           15836 non-null  object
 2   date              15492 non-null  object
 3   transaction_type  15836 non-null  object
 4   category          15678 non-null  object
 5   amount            15658 non-null  object
 6   payment_mode      15333 non-null  object
 7   location          15114 non-null  object
 8   notes             14302 non-null  object
dtypes: object(9)
memory usage: 1.1+ MB


### Preview rows (At random)

In [5]:
df_raw.sample(10, random_state=42)

,transaction_id,user_id,date,transaction_type,category,amount,payment_mode,location,notes
13342,T13747,U140,21-01-20,Expense,Food,$199,UPI,Lucknow,Paid electricity bill
169,T02374,U089,06/02/2020,Expense,Rent,106,UPI,Pune,Doctor visit
2099,T04501,U001,September 15 2021,Expense,Rent,"₹1,017",Cash,Jaipur,Netflix subscription
4948,T00737,U093,10/13/2020,Expense,Rent,455,Cash,MUMBAI,NaN
5771,T03866,U088,16-12-21,Expense,Rent,449,UPI,NaN,Paid electricity bill
8494,T12657,U013,06/17/2019,Expense,Entertainment,300,Card,Pune,Netflix subscription
14961,T02693,U072,08-08-20,Expense,Others,297,UPI,Ahmedabad,Dinner at resto cash
8035,T10026,U165,27-06-22,Income,Salary,"62,768",Card,Jaipur,Monthly rent via app
88,T11740,U083,05/20/2021,Expense,Food,"1,766",Cash,Chennai,Grocery shopping
12179,T13009,U077,February 02 2020,Expense,Rent,₹818,UPI,Bengaluru,Paid electricity bill


### Missing Values

In [6]:
df_raw.isna().sum().sort_values(ascending=False)

notes               1534
location             722
payment_mode         503
date                 344
amount               178
category             158
transaction_id         0
user_id                0
transaction_type       0
dtype: int64

### Unique Values

In [7]:
df_raw[['transaction_type', 'category', 'payment_mode']].nunique()

transaction_type      2
category            212
payment_mode         62
dtype: int64

## Data Audit Summary

The dataset contains **15,836 rows** and **9 columns**, representing individual financial transactions across multiple users. All columns are stored as `object` type at this stage, which is expected before cleaning.

### Missing Data Overview
A preliminary inspection shows varying levels of missing values:

- **notes:** 1,534 missing  
- **location:** 722 missing  
- **payment_mode:** 503 missing  
- **date:** 344 missing  
- **amount:** 178 missing  
- **category:** 158 missing  
- `transaction_id`, `user_id`, and `transaction_type` have no missing values.

This pattern suggests that the dataset is moderately messy, with missing contextual fields such as location and notes, and some critical analytical fields ("date" and "amount") that must be cleaned.

### Data Type Issues
All columns are currently of type `object`, which is typical for raw CSV data. However:

- **date** contains a mixture of formats (e.g., `"September 15 2021"`, `"21-01-20"`, `"06/02/2020"`, `"16-12-21"`), meaning a custom multi-format parser will be required.
- **amount** values contain mixed formatting such as currency symbols (`₹`, `$`), commas (`"62,768"`), and plain numbers (`"998"`). These must be cleaned and converted to numeric.
- **payment_mode** and **category** show inconsistent levels of granularity and capitalization.

### Unique Value Checks
- **transaction_type:** 2 values (`Income`, `Expense`) – consistent.
- **category:** 212 unique categories – unusually high, suggesting inconsistent labeling.
- **payment_mode:** 62 unique values – indicates mixed formatting and possible misspellings.

### Key Findings
- The dataset requires **significant transformation**, especially in the `date`, `amount`, `category`, and `payment_mode` fields.
- The structural integrity is good (no missing user or transaction IDs).
- After cleaning, this dataset will be suitable for analysis such as spending trends, category insights, and user-level financial behavior.

## Transform Stage

### Parsing Dates

In [8]:
#  First-pass date parsing

def parse_dates(series):
    return pd.to_datetime(
        series,
        errors='coerce',
        infer_datetime_format=True
    )

df_raw['date_parsed'] = parse_dates(df_raw['date'])

df_raw[['date', 'date_parsed']].head(20)

/var/folders/kw/410yfyf16jl9_7z_msy08s900000gn/T/ipykernel_89400/3125422801.py:4: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.to_datetime(


,date,date_parsed
0,December 22 2021,2021-12-22
1,03/24/2022,NaT
2,October 18 2022,2022-10-18
3,12/12/2021,NaT
4,NaN,NaT
5,12-07-22,NaT
6,September 12 2019,2019-09-12
7,2022-01-06,NaT
8,04-11-22,NaT
9,February 03 2021,2021-02-03


In [9]:
# Identifying how many dates failed to parse
df_raw['date_parsed'].isna().sum()

np.int64(13486)

### Date Parsing Attempt 1 – Results

The first parsing attempt using `pd.to_datetime(..., infer_datetime_format=True)` succeeded only on long-form dates such as:

- "December 22 2021"
- "October 18 2022"
- "September 12 2019"

However, it failed on most numeric formats, including:

- `MM/DD/YYYY` (e.g., "03/24/2022")
- `DD/MM/YYYY` or `DD-MM-YY` (e.g., "12-07-22", "21-01-20")
-  Short dates (e.g., "2022-01-06")

Out of **15,836** records, **13,486** dates failed to parse on the first attempt. This confirms that the dataset includes multiple date patterns.

In [10]:
import re
from datetime import datetime

# A list of patterns we want to try manually
date_formats = [
    "%m/%d/%Y",  # 03/24/2022
    "%d/%m/%Y",  # 24/03/2022
    "%Y-%m-%d",  # 2022-01-06
    "%d-%m-%y",  # 12-07-22
    "%d-%m-%Y",  # 12-07-2022
    "%m-%d-%y",  # 07-12-22
]

def parse_with_formats(date_str):
    if pd.isna(date_str):
        return None
    
    # Try each known format
    for fmt in date_formats:
        try:
            return datetime.strptime(date_str, fmt)
        except:
            pass
    
    return None


# Apply the custom parser ONLY to rows where the first parser failed
mask_failed = df_raw['date_parsed'].isna()

df_raw.loc[mask_failed, 'date_parsed'] = df_raw.loc[mask_failed, 'date'].apply(parse_with_formats)

df_raw[['date', 'date_parsed']].head(20)

,date,date_parsed
0,December 22 2021,2021-12-22
1,03/24/2022,2022-03-24
2,October 18 2022,2022-10-18
3,12/12/2021,2021-12-12
4,NaN,NaT
5,12-07-22,2022-07-12
6,September 12 2019,2019-09-12
7,2022-01-06,2022-01-06
8,04-11-22,2022-11-04
9,February 03 2021,2021-02-03


### Date Parsing Attempt 2 – Results

The date parser now successfully handles the remaining numeric date formats.

In [11]:
# Check parsing improvement
print(f"Failed to parse after attempt 2: {df_raw['date_parsed'].isna().sum():,}")
print(f"Successfully parsed: {(~df_raw['date_parsed'].isna()).sum():,}")
print(f"Success rate: {((~df_raw['date_parsed'].isna()).sum() / len(df_raw)) * 100:.1f}%")

# Look at remaining unparsed dates
remaining_unparsed = df_raw[df_raw['date_parsed'].isna() & df_raw['date'].notna()]['date'].unique()
print(f"\nRemaining unparsed date patterns ({len(remaining_unparsed)}):")
print(remaining_unparsed[:10] if len(remaining_unparsed) > 0 else "None")

Failed to parse after attempt 2: 381
Successfully parsed: 15,455
Success rate: 97.6%

Remaining unparsed date patterns (36):
['29/10/19' '2020/11/17' '2020/05/05' '2022/12/09' '28/09/22' '2020/02/20'
 '21/08/21' '11/02/22' '2021/02/16' '06/07/20']


### Cleaning Amount Values

Now we need to handle the mixed currency formatting in the amount column. From our inspection, we saw values like `$143`, `₹1,017`, `62,768`, and plain numbers like `998`.

In [12]:
# Examine unique amount patterns before cleaning
print("Sample amount values before cleaning:")
print(df_raw['amount'].dropna().sample(20, random_state=42).tolist())

# Function to clean amount values
def clean_amount(amount_str):
    if pd.isna(amount_str):
        return None
    
    # Convert to string and remove currency symbols and commas
    cleaned = str(amount_str).replace('₹', '').replace('$', '').replace(',', '').strip()
    
    try:
        return float(cleaned)
    except:
        return None

# Apply cleaning
df_raw['amount_cleaned'] = df_raw['amount'].apply(clean_amount)

print(f"\nAmount cleaning results:")
print(f"Original non-null amounts: {df_raw['amount'].notna().sum():,}")
print(f"Successfully cleaned amounts: {df_raw['amount_cleaned'].notna().sum():,}")
print(f"Failed to clean: {(df_raw['amount'].notna() & df_raw['amount_cleaned'].isna()).sum():,}")

# Check conversion results
print(f"\nAmount statistics after cleaning:")
df_raw['amount_cleaned'].describe()

Sample amount values before cleaning:
['124', '$406', '410', '267', '231', '1,610', '314', '1,554', '891', '$323', '846', '71', '393', '653', '₹409', '2,883', '447', '247', '1,517', '1,434']

Amount cleaning results:
Original non-null amounts: 15,658
Successfully cleaned amounts: 15,658
Failed to clean: 0

Amount statistics after cleaning:


count     15658.000000
mean      12477.195491
std       56249.380633
min       -1313.000000
25%         203.000000
50%         534.000000
75%        1742.000000
max      999999.000000
Name: amount_cleaned, dtype: float64

### Cleaning Other Categorical Fields

 Standardizing the categorical fields that showed inconsistencies.

In [13]:
# Clean payment_mode field
print("Payment mode values before cleaning:")
print(f"Unique values: {df_raw['payment_mode'].nunique()}")
print(df_raw['payment_mode'].value_counts().head(10))

# Standardize payment modes
def clean_payment_mode(mode):
    if pd.isna(mode):
        return 'Unknown'
    
    mode = str(mode).strip().title()
    
    # Standardize variations
    if 'bank' in mode.lower() or 'transfer' in mode.lower():
        return 'Bank Transfer'
    elif mode.lower() in ['upi', 'paytm', 'gpay', 'phonepe']:
        return 'UPI'
    elif mode.lower() in ['card', 'debit', 'credit']:
        return 'Card' 
    elif mode.lower() == 'cash':
        return 'Cash'
    else:
        return mode  # Keep other values as-is

df_raw['payment_mode_cleaned'] = df_raw['payment_mode'].apply(clean_payment_mode)

print(f"\nPayment mode after cleaning:")
print(f"Unique values: {df_raw['payment_mode_cleaned'].nunique()}")
print(df_raw['payment_mode_cleaned'].value_counts())

# Clean categories - group similar ones
print(f"\n\nCategory values before cleaning:")
print(f"Unique values: {df_raw['category'].nunique()}")
print(df_raw['category'].value_counts().head(10))

Payment mode values before cleaning:
Unique values: 62
payment_mode
Bank Transfer    3838
Cash             3787
UPI              3736
Card             3721
UI                 15
Csah               12
Cah                11
PUI                10
UPPI                9
UP                  9
Name: count, dtype: int64

Payment mode after cleaning:
Unique values: 35
payment_mode_cleaned
Bank Transfer    3902
Cash             3787
UPI              3736
Card             3721
Unknown           503
Ui                 15
Csah               12
Cah                11
Pui                10
Uppi                9
Up                  9
Uupi                8
Pi                  8
Car                 8
Ash                 8
Cassh               8
Ard                 8
Caard               7
Crad                7
Acsh                6
Crd                 6
Cadr                6
Uip                 5
Upii                5
Cardd               4
Csh                 4
Cashh               4
Carrd               3
A

### Adding Calculated Fields (Feature Engineering)


In [18]:
# Create working dataframe with cleaned data for calculations
df_clean = df_raw.copy()

# Replace original columns with cleaned versions
df_clean['date'] = df_clean['date_parsed']
df_clean['amount'] = df_clean['amount_cleaned'] 
df_clean['payment_mode'] = df_clean['payment_mode_cleaned']

# Remove rows with missing critical data
df_clean = df_clean.dropna(subset=['amount'])
print(f"Dataset after removing missing amounts: {df_clean.shape}")

# 1. Transaction frequency per user
user_transaction_counts = df_clean.groupby('user_id')['transaction_id'].count()
df_clean['user_transaction_frequency'] = df_clean['user_id'].map(user_transaction_counts)

# 2. Total spending per user
user_total_spending = df_clean.groupby('user_id')['amount'].sum()
df_clean['user_total_spending'] = df_clean['user_id'].map(user_total_spending)

# 3. Average transaction amount per user  
user_avg_amount = df_clean.groupby('user_id')['amount'].mean()
df_clean['user_avg_transaction'] = df_clean['user_id'].map(user_avg_amount)

# 4. Expense ratio per user (expenses / total transactions)
user_expense_counts = df_clean[df_clean['transaction_type'] == 'Expense'].groupby('user_id')['transaction_id'].count()
df_clean['user_expense_ratio'] = df_clean['user_id'].map(user_expense_counts / user_transaction_counts).fillna(0)

# 5. Add temporal features for valid dates
df_clean['year'] = df_clean['date'].dt.year
df_clean['month'] = df_clean['date'].dt.month
df_clean['day_of_week'] = df_clean['date'].dt.dayofweek
df_clean['is_weekend'] = df_clean['day_of_week'].isin([5, 6]).astype(int)

# 6. Transaction amount categories
df_clean['amount_category'] = pd.cut(df_clean['amount'], 
                                   bins=[0, 100, 500, 1000, 5000, float('inf')],
                                   labels=['Small', 'Medium', 'Large', 'Very Large', 'Extreme'])

print("Calculated fields added:")
print("- user_transaction_frequency: Number of transactions per user")
print("- user_total_spending: Total amount spent per user") 
print("- user_avg_transaction: Average transaction amount per user")
print("- user_expense_ratio: Ratio of expenses to total transactions per user")
print("- year, month, day_of_week, is_weekend: Temporal features")
print("- amount_category: Categorized transaction amounts")

print(f"Final dataset shape: {df_clean.shape}")
print(f"Final columns: {list(df_clean.columns)}")

Dataset after removing missing amounts: (15658, 12)
Calculated fields added:
- user_transaction_frequency: Number of transactions per user
- user_total_spending: Total amount spent per user
- user_avg_transaction: Average transaction amount per user
- user_expense_ratio: Ratio of expenses to total transactions per user
- year, month, day_of_week, is_weekend: Temporal features
- amount_category: Categorized transaction amounts
Final dataset shape: (15658, 21)
Final columns: ['transaction_id', 'user_id', 'date', 'transaction_type', 'category', 'amount', 'payment_mode', 'location', 'notes', 'date_parsed', 'amount_cleaned', 'payment_mode_cleaned', 'user_transaction_frequency', 'user_total_spending', 'user_avg_transaction', 'user_expense_ratio', 'year', 'month', 'day_of_week', 'is_weekend', 'amount_category']


### Final Data Validation and Export

Inspecting the cleaned dataset and exporting it for use in the later stages.

In [20]:
# Final validation
print("=== FINAL DATASET VALIDATION ===")
print(f"Final shape: {df_clean.shape}")
print(f"Original raw dataset: {df_raw.shape}")
print(f"Data reduction: {((df_raw.shape[0] - df_clean.shape[0]) / df_raw.shape[0]) * 100:.1f}%")

print("Missing values in final dataset:")
missing_summary = df_clean.isnull().sum().sort_values(ascending=False)
print(missing_summary[missing_summary > 0])

print("Data types in final dataset:")
print(df_clean.dtypes)

print("Sample of final cleaned data:")
df_clean.head()

=== FINAL DATASET VALIDATION ===
Final shape: (15658, 21)
Original raw dataset: (15836, 12)
Data reduction: 1.1%
Missing values in final dataset:
notes              1523
location            713
date                376
day_of_week         376
month               376
date_parsed         376
year                376
category            154
amount_category      62
dtype: int64
Data types in final dataset:
transaction_id                        object
user_id                               object
date                          datetime64[ns]
transaction_type                      object
category                              object
amount                               float64
payment_mode                          object
location                              object
notes                                 object
date_parsed                   datetime64[ns]
amount_cleaned                       float64
payment_mode_cleaned                  object
user_transaction_frequency             int64
user_total_

,transaction_id,user_id,date,transaction_type,category,amount,payment_mode,location,notes,date_parsed,amount_cleaned,payment_mode_cleaned,user_transaction_frequency,user_total_spending,user_avg_transaction,user_expense_ratio,year,month,day_of_week,is_weekend,amount_category
0,T03512,U039,2021-12-22,Expense,Rent,998.0,Cash,Pune,Paid electricity bill,2021-12-22,998.0,Cash,86,1017618.0,11832.767442,0.825581,2021.0,12.0,2.0,0,Large
1,T03261,U179,2022-03-24,Expense,Food,143.0,Card,Delhi,Grocery shopping,2022-03-24,143.0,Card,87,945163.0,10863.942529,0.839080,2022.0,3.0,3.0,0,Medium
2,T04316,U143,2022-10-18,Expense,Rent,149.0,Cash,Bengaluru,NaN,2022-10-18,149.0,Cash,90,876727.0,9741.411111,0.855556,2022.0,10.0,1.0,0,Medium
3,T05649,U079,2021-12-12,Expense,Rent,49.0,UPI,NaN,Paid electricity bill,2021-12-12,49.0,UPI,82,706444.0,8615.170732,0.853659,2021.0,12.0,6.0,1,Small
4,T14750,U020,NaT,Income,Other Income,83802.0,Bank Transfer,Chennai,Gift via app,NaT,83802.0,Bank Transfer,101,1108349.0,10973.752475,0.831683,NaN,NaN,NaN,0,Extreme


In [22]:
# Select relevant columns for final export
final_columns = [
    'transaction_id', 'user_id', 'date', 'transaction_type', 'category', 
    'amount', 'payment_mode', 'location', 'notes',
    'user_transaction_frequency', 'user_total_spending', 'user_avg_transaction', 
    'user_expense_ratio', 'year', 'month', 'day_of_week', 'is_weekend', 
    'amount_category'
]

df_final = df_clean[final_columns].copy()

# Export to transformed data folder
output_path = "../data/transformed/cleaned_dataset.csv"
df_final.to_csv(output_path, index=False)

print(f"=== DATASET EXPORTED ===")
print(f"Saved to: {output_path}")
print(f"Final dataset dimensions: {df_final.shape}")
print(f"Columns exported: {list(df_final.columns)}")


=== DATASET EXPORTED ===
Saved to: ../data/transformed/cleaned_dataset.csv
Final dataset dimensions: (15658, 18)
Columns exported: ['transaction_id', 'user_id', 'date', 'transaction_type', 'category', 'amount', 'payment_mode', 'location', 'notes', 'user_transaction_frequency', 'user_total_spending', 'user_avg_transaction', 'user_expense_ratio', 'year', 'month', 'day_of_week', 'is_weekend', 'amount_category']
